# Absorption correction
The transmission (absorption) calculation used here is a home-cooked variation of the equations presented in *He, Bob B. Two-dimensional X-ray Diffraction. John Wiley & Sons, 2018* **p.203-207**.  
The beam path length is the sum of the incident and diffracted beam path length:
$$ t = \frac{z}{\cos{\eta}} + \frac{t_0-z}{\cos{\zeta}} $$
where $t_0$ is the thickness of the (flat) sample, $z$ is the distance from the surface, $\cos{\eta}$ is the dot product of incident beam and the sample normal, and $\cos{\zeta}$ is the dot product of the diffracted beam and the sample normal. (All vectors are unit vectors)  
The transmitted is then the integral of $e^{-\mu\cdot t}$ over the sample thickness $t_0$, normalized with the incident path length.

##### Note on DanMAX coordinate definition  
The coordinates are here defined from eulerian angles: ω χ, and φ, using YXY rotations. The laboratory coordinates X<sub>L</sub>, Y<sub>L</sub>, and Z<sub>L</sub> are defined such that Z<sub>L</sub> is along the beam,  Y<sub>L</sub> is vertical with the positive direction upwards and  X<sub>L</sub> is horizontal, pointing away from the ring. Consequently, ω is a rotation in the horizontal plane around Y<sub>L</sub>, χ is a rotation around the new X<sub>L</sub>', and φ is a rotation around Y<sub>L</sub>''.  
Example 1:  
A sample normal along the beam has the (x,y,z) coordinates (0,0,1). Rotating ω 90° would result in the new coordinates (x',y',z') = (1,0,0).  
Example 2:  
A sample normal along the beam rotated along all three angles 90° results in the following new corrdinates: (0,0,1) -> (1,0,0) -> (0,-1,0) -> (0,0,-1)  
  
NB: while the sample normal in example 2 ends up anti-parallel with the beam, it is *not* the same as a 180° ω-rotation as the sample will also have rotated around the normal. 

In [ ]:
%matplotlib widget
import sys
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
#To import DanMAX from the folder above:
sys.path.append('../')
import DanMAX as DM
from DanMAX import texture as tx #.absorptionCorrection1D, absorptionCorrection2D
style = DM.darkMode(style_dic={'figure.figsize':'large'})

## 1D absorption correction

In [ ]:
fname = DM.findScan()
aname = DM.getAzintFname(fname)

data = DM.getAzintData(aname)
if type(data['q']) != type(None):
    x = data['q']
    meta = DM.getMetaData(fname)
    tth = DM.Q2tth(x,meta['energy'])
else:
    tth = data['tth']
I = data['I']

In [ ]:
# user-provided absorption coefficient at the relevant energy
mu =  38.31  # cm-1 Fe @35 keV
# user-provided sample thickness in mm
t0 = 0.2

A_1d = tx.absorptionCorrection1D(tth,t0,mu,normalize=True)

In [ ]:
fig, [ax0,ax1] = plt.subplots(2,sharex=True)
ax0.set_title('Normalized absorption correction')
ax0.plot(tth,A_1d)

ax1.set_title('Average diffraction patterns')
ax1.plot(tth,np.mean(I,axis=0),label='Original')
ax1.plot(tth,np.mean(I,axis=0)*A_1d,label='Corrected')

ax1.legend()
ax0.set_ylabel('Correction')
ax1.set_xlabel('2theta (deg)')
ax1.set_ylabel('Intensity (a.u.)')

## 2D absorption correction
Calculate the 2D absorption correction based on the geometry of a user-provided PONI file

In [ ]:
fname = DM.findScan(400)
im = DM.getAverageImage(fname)

#with h5.File(fname,'r') as f:
#    im = f['/entry/instrument/pilatus/data'][:]

In [ ]:
# user-provided .poni file path
pname = '/data/visitors/danmax/PROPOSAL/VISIT/process/USERPONI.poni'
# user-provided absorption coefficient at the relevant energy
mu =  38.31  # cm-1 Fe @35 keV
# user-provided sample thickness in mm
t0 = .2
# sample rotation (YXY)
omega,chi,phi = 0,0,0

A_2d = tx.absorptionCorrection2D(pname,t0,mu,omega,chi,phi,normalize=True)

In [ ]:
fig, axes = plt.subplots(1,3,sharex=True,sharey=True)
ax0,ax1,ax2 = axes
for ax in axes:
    ax.grid(False)
    ax.set_xticks([])
    ax.set_yticks([])

ax0.set_title('Absorption correction')
ax0.imshow(A_2d)

ax1.set_title('Original')
cm_og = ax1.imshow(im,norm='log')

ax2.set_title('Corrected')
cm_corr = ax2.imshow(im*A_2d,norm='log')
# set the same color-limits for the two images
cm_og.set_clim(cm_corr.get_clim())